This notebook demonstrates PPG waveform fitting.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal, optimize
from pyampd import ampd

import sys
sys.path.append('../')
from src  import synthetic_ppg, model_parameters, utils

plt.style.use('./article.mplstyle')

In [ ]:
# Use default model parameters.
model_params = model_parameters.ModelParameters()

# Load an example raw green PPG signal.
ppg = np.load('../data/measurements/raw_green_ppg_signal_for_waveform_fitting.npy', allow_pickle=True)

# Flipping, filtering and normalization.
ppg_filt = utils.min_max_normalize(signal.sosfiltfilt(model_params.sos, -ppg), 0, 1)

# Feet.
feet = ampd.find_peaks(-ppg_filt)
# Nth foot index.
foot_idx = 9
# Waveform.
wf = utils.min_max_normalize(ppg_filt[feet[foot_idx]:feet[foot_idx + 1]])
# Pulse width.
wf_pulse_width = [feet[foot_idx + 1] - feet[foot_idx]]

fig, ax = plt.subplots()
fig.set_size_inches(8.7 / 2.54, 8.7 / 2.54)
ax.plot(wf)
ax.spines[['right', 'top']].set_visible(False)

In [ ]:
def objective_n_gaussians(coeffs, fs, pulse_widths, s_real, N):
    """
    Objective function for finding the optimal synthetic PPG 
    model parameters.
    """
    locs = coeffs[:N]
    widths = coeffs[N:2 * N]
    amps = coeffs[2 * N:3*N]
    s_synt, _, _, _, _ = synthetic_ppg.gen_ppg(locs, widths, amps, pulse_widths, fs=fs)

    return np.array((s_synt - s_real) ** 2).sum()

# Number of Gaussian bumps.
N = 2
locs_bounds = (-np.pi, np.pi)
sigmas_bounds = (0, 3)
amps_bounds = (0, 10)

# Parameter boundaries.
bounds = []
for _ in range(N):
    bounds.append(locs_bounds)
for _ in range(N):
    bounds.append(sigmas_bounds)
for _ in range(N):
    bounds.append(amps_bounds)
print(f'Bounds: {bounds}')

# Constraints.
cons_matrix = []
# Location constraints. The first Gaussian bumps is the rightmost etc.
loc_cons = np.zeros(N * 3)
loc_cons[0] = 1
loc_cons[1] = -1
for i in range(N - 1):
    cons_matrix.append(np.roll(loc_cons, i))
# Width constraints. The first Gaussian bumps is the narrowest.
width_cons = np.zeros(N * 3)
width_cons[N] = 1
width_cons[N + 1] = -1
for i in range(N - 1):
    cons_matrix.append(np.roll(width_cons, i))

# The final matrix.
cons_matrix = np.array(cons_matrix)
print(f'Constaints matrix: {cons_matrix}')
cons = optimize.LinearConstraint(cons_matrix, -np.Inf, 0)

# Run optimization.
res = optimize.differential_evolution(objective_n_gaussians, bounds, args=(model_params.fs, wf_pulse_width, wf, N), constraints=cons)
# Optimal parameters.
coeffs = res.x
print(f'Coefficients: {coeffs}')

In [ ]:
# Plot the optimal synthetic signal together with the real signal.
locs = coeffs[:N]
widths = coeffs[N:2 * N]
amps = coeffs[2 * N:3*N]
synt, label_feet, label_peaks, der, der_raw = synthetic_ppg.gen_ppg(locs, widths, amps, wf_pulse_width, model_params.fs)

def plot_real_and_synthetic_waveforms(ts_s, wf, synt, der):

    fig, axes = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    axes = axes.flatten()
    fig.set_size_inches(8.7 / 2.54, 8.7 / 2.54)

    axes[0].plot(ts_s, wf, color='tab:orange', label='Real', linewidth=1)
    axes[0].plot(ts_s, synt, color='tab:blue', label='Synthetic', linewidth=1)
    # axes[0].plot(ts_s, s_label, color='tab:red', label='Label', linewidth=1)

    axes[1].plot(ts_s, utils.min_max_normalize(np.gradient(wf)), color='tab:orange', linewidth=1)
    axes[1].plot(ts_s, utils.min_max_normalize(der), color='tab:blue', linewidth=1)

    y_ticks = [0, 0.5, 1]
    y_ticks_str = ['0.0', '0.5', '1.0']
    for i in range(2):
        if i == 1:
            axes[i].set_xlabel('Time [s]')
        # axes[i].set_ylabel('[arb. unit]')
        axes[i].set_yticks(y_ticks, y_ticks_str)
        axes[i].spines[['right', 'top']].set_visible(False)

    axes[0].set_title('Pulse waveforms', pad=3)
    axes[1].set_title('First derivatives', pad=3)

    fig.supylabel('Normalized amplitude', x=-0.01, y=0.5)

    fig.subplots_adjust(bottom=0.2)
    fig.legend(*fig.axes[0].get_legend_handles_labels(), ncol=2, bbox_to_anchor=(0.5, 0.04), loc='center', frameon=False)
    
ts_s = np.arange(wf.size) / model_params.fs
plot_real_and_synthetic_waveforms(ts_s, wf, synt, der)

Plot the same but now using the saved parameter values.

In [ ]:
synt, label_feet, label_peaks, der, der_raw = synthetic_ppg.gen_ppg(
    model_params.gaus_locs_fitted, model_params.gaus_widths_fitted, 
    model_params.gaus_amps_fitted, wf_pulse_width, model_params.fs)

ts_s = np.arange(wf.size) / model_params.fs
plot_real_and_synthetic_waveforms(ts_s, wf, synt, der)

Plot waveforms using the min and max parameter ranges.

In [ ]:
pulse_widths = [100]

synt_fitted, _, _, _, _ = synthetic_ppg.gen_ppg(
    model_params.gaus_locs_fitted, model_params.gaus_widths_fitted, 
    model_params.gaus_amps_fitted, pulse_widths
)
synt_min, _, _, _, _ = synthetic_ppg.gen_ppg(
    [model_params.gaus_loc_bounds[0][0], model_params.gaus_loc_bounds[1][0]], 
    [model_params.gaus_width_bounds[0][0], model_params.gaus_width_bounds[1][0]], 
    [model_params.gaus_amp_bounds[0][0], model_params.gaus_amp_bounds[1][0]], 
    pulse_widths)
synt_max, _, _, _, _ = synthetic_ppg.gen_ppg(
    [model_params.gaus_loc_bounds[0][1], model_params.gaus_loc_bounds[1][1]], 
    [model_params.gaus_width_bounds[0][1], model_params.gaus_width_bounds[1][1]], 
    [model_params.gaus_amp_bounds[0][1], model_params.gaus_amp_bounds[1][1]], 
    pulse_widths)
synt_mean, _, _, _, _ = synthetic_ppg.gen_ppg(
    [np.mean(model_params.gaus_loc_bounds[0]), np.mean(model_params.gaus_loc_bounds[1])], 
    [np.mean(model_params.gaus_width_bounds[0]), np.mean(model_params.gaus_width_bounds[1])], 
    [np.mean(model_params.gaus_amp_bounds[0]), np.mean(model_params.gaus_amp_bounds[1])], 
    pulse_widths)

ts_s = np.arange(0, pulse_widths[0]) / model_params.fs

fig, ax = plt.subplots()
fig.set_size_inches(5 / 2.54, 5 / 2.54)
ax.plot(ts_s, utils.min_max_normalize(synt_fitted), color='k',
    linewidth=1, label='Fitted')
ax.plot(ts_s, utils.min_max_normalize(synt_min), color='tab:blue', 
    linewidth=1, label='Minimum')
ax.plot(ts_s, utils.min_max_normalize(synt_mean), color='tab:orange', 
    linewidth=1, label='Mean')
ax.plot(ts_s, utils.min_max_normalize(synt_max), color='tab:green', 
    linewidth=1, label='Maximum')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Normalized amplitude')
# ax.legend(frameon=False)
ax.spines[['right', 'top']].set_visible(False)
fig.tight_layout()
fig.subplots_adjust(bottom=0.35)
fig.legend(*fig.axes[0].get_legend_handles_labels(), ncol=2, bbox_to_anchor=(0.5, 0.04), loc='center', frameon=False)

In [ ]:
n = 100
fs = 100
pulse_widths = [fs]
shifts = np.linspace(0, 1, n)
waveforms = np.zeros((n, pulse_widths[0]))
for i, sh in enumerate(shifts):
    locs = [item[0] + sh * (item[1] - item[0]) for item in model_params.gaus_loc_bounds]
    widths = [item[0] + sh * (item[1] - item[0]) for item in model_params.gaus_width_bounds]
    amps = [item[0] + sh * (item[1] - item[0]) for item in model_params.gaus_amp_bounds]
    waveforms[i, :], _, _, _, _ = synthetic_ppg.gen_ppg(locs, widths, amps, pulse_widths, fs)

fig = plt.figure()
fig.set_size_inches(5 / 2.54, 5 / 2.54)
ax = fig.add_subplot(111, projection='3d')
x, y = np.meshgrid(np.arange(pulse_widths[0]) / fs, np.arange(n))
ax.plot_surface(x, y, waveforms, rstride=1, cstride=1, cmap=matplotlib.cm.jet, linewidth=0, antialiased=False)
# ax.margins(0)
ax.grid(False)
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.set_xlabel('Time [s]', labelpad=0)
ax.set_ylabel('Waveform [#]', labelpad=0)
ax.set_zlabel('Normalized amplitude', labelpad=0)
ax.tick_params(axis='both', which='major', pad=0)
fig.set_tight_layout(True)